In [ ]:
pip install --user mlflow

In [1]:
import pandas as pd
import numpy as np
import string
import xlrd
import mlflow
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline

In [2]:
train = pd.ExcelFile("CAC_ knowledge.xlsx")
df = train.parse(train.sheet_names[0])

In [3]:
df

,Issue Description,Solution/Workaround
0,Intermittent issue in triggering child policy,"In policy management, while setting up sequent..."
1,NLP mapping issue,Mapping was not done due to some special chara...
2,Intermittent Snow Base action failure,the issue was due to user specific connectivi...
3,authentication error while downloading the fil...,the issue was solved by passing the JWT token ...
4,Acknowledgement email not received for the Act...,removed the workplace user and once again adde...
5,Additional Attribute is not getting set for ex...,validate the additional attribute key name is ...
6,Additional attribute not mapped when the actio...,The custom script used in child workflow is re...
7,Virtual envs libraries issue,if there is any modules issue after any patch ...
8,Credentials are miss-matching while we run the...,Check for the derived action credential and al...
9,Handling unix broot privelleges,This feature has been handled by a custom work...


In [4]:
df.shape

(34, 2)

In [5]:
df = df.dropna()
df.shape

(34, 2)

In [6]:
df = df.drop_duplicates(subset='Issue Description')
df.shape

(34, 2)

In [7]:
def ask_question(question, tfidf_matrix):
    query_vect = tfidf_vectorizer.transform([question])
    similarity = cosine_similarity(query_vect, tfidf_matrix)
    max_similarity = np.argmax(similarity, axis=None)
    sim_percentage = 0.20
    index = 1
    output = "Closest found Issues and Solutions\n"
    for m in range(len(similarity)):
        for n in range(len(similarity[m])):
            if similarity[m][n] >= sim_percentage:
                output += str(index) + ". " + df.iloc[n]['Issue Description'] + " -> " + df.iloc[n]['Solution/Workaround'] + "\n"
                index = index+1
    return output

In [9]:
from run_prediction_ticket import log_model
from run_prediction_ticket import TicketClassifierPyfunc
with mlflow.start_run():
    know_base = df.to_dict('records')
    result_dict={}
    for i in know_base:
        result_dict[i['Issue Description']] = i["Solution/Workaround"]
        
    #print(result_dict)
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(tuple(df['Issue Description']))
    # Set the modelname
    #mlflow.set_tag("modelname", 'Ticket_solution_model')
    log_model(tfidf_matrix,tfidf_vectorizer,result_dict, "model","Ticket_solution_model")
    val = input("Enter your issue : ")
    #output = ask_question(model_pred,val, tfidf_matrix)
    output = TicketClassifierPyfunc.model_pred(tfidf_matrix, tfidf_vectorizer, result_dict, df)
    print(output)
    print(mlflow.active_run().info.run_id)

loader_module %s run_prediction_ticket
datapath %s C:\Users\733600\AppData\Local\Temp\1\tmpk_qlj0v_\Ticket_solution_model
Enter your issue : actions are failing
*****Model prediction*******
Closest found Issues and Solutions
1. Intermittent issue in triggering child policy  -> In policy management, while setting up sequential actions ,there is a new feature called "Standalone" .Please select that option whenever you are setting up action in policy. If your action is dependent on any child policies please select it to be "No" or else "yes".
As the collab engine has a batch that checks for policy against each action , setting up this option will reduce the time and also the load of the collab engine to individually check every action.It also reduces the chance of failure. 
2. Intermittent Snow Base action failure -> the issue was due to  user specific connectivity issue or an intermittant issue and can be solved by checking the connection or executing the action again
3.  child policy is